In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from konlpy.tag import Okt

In [2]:
pd.set_option('display.unicode.east_asian_width',True) #행열 정렬

In [3]:
df = pd.read_csv('./data/raw_65548_2021-06-22.csv', engine='python')
print(df.head())
print(df.info())

                         title  \
0    황성주 박사의 생식과 건강   
1  아름다운 청년, 대니 서의 집   
2           여왕 패션 종이인형   
3       더 포스터 북 by 이미소   
4                    가죽 소품   

                                                                                summary  \
0  생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레...   
1  환경 운동가 대니 서가 공개하는 환경 친화적인 집 꾸미기. 이 책은 교외에 위치한 ...      
2  다양한 업적을 가진 세계의 여왕 패션을 직접 입혀보고, 아름답고 다양한 드레스와 왕...    
3  직관적인 제목처럼 권마다 포스터 작품 10점으로 가득 채워진 도서이다. 그러나 일반...     
4  가죽공예의 기본 테크닉에서부터 명함 지갑, 동전 지갑, 물림쇠 지갑, 파우치, 안경...      

           category  
0  Home_Cook_Beauty  
1  Home_Cook_Beauty  
2  Home_Cook_Beauty  
3  Home_Cook_Beauty  
4  Home_Cook_Beauty  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65548 entries, 0 to 65547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     65548 non-null  object
 1   summary   65516 non-null  object
 2   category  65548 non-null  object
dtypes: object(3)
memory u

In [5]:
df= df.dropna(axis=0)
print(df.head())
print(df.info())

                         title  \
0    황성주 박사의 생식과 건강   
1  아름다운 청년, 대니 서의 집   
2           여왕 패션 종이인형   
3       더 포스터 북 by 이미소   
4                    가죽 소품   

                                                                                summary  \
0  생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레...   
1  환경 운동가 대니 서가 공개하는 환경 친화적인 집 꾸미기. 이 책은 교외에 위치한 ...      
2  다양한 업적을 가진 세계의 여왕 패션을 직접 입혀보고, 아름답고 다양한 드레스와 왕...    
3  직관적인 제목처럼 권마다 포스터 작품 10점으로 가득 채워진 도서이다. 그러나 일반...     
4  가죽공예의 기본 테크닉에서부터 명함 지갑, 동전 지갑, 물림쇠 지갑, 파우치, 안경...      

           category  
0  Home_Cook_Beauty  
1  Home_Cook_Beauty  
2  Home_Cook_Beauty  
3  Home_Cook_Beauty  
4  Home_Cook_Beauty  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 65516 entries, 0 to 65547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     65516 non-null  object
 1   summary   65516 non-null  object
 2   category  65516 non-null  object
dtypes: object(3)
memory u

In [7]:
col_dup = df[['title','summary']].duplicated()
print(col_dup)
sum_dup = df[['title','summary']].duplicated().sum()
print(sum_dup)

0        False
1        False
2        False
3        False
4        False
         ...  
65543     True
65544    False
65545    False
65546     True
65547    False
Length: 65516, dtype: bool
4372


In [9]:
df = df.drop_duplicates(subset=['title','summary'])
sum_dup = df[['title','summary']].duplicated().sum()
print(sum_dup) #중복된 개수의 합

0


In [10]:
#인덱스를 처음부터 다시 붙여주는것
df.reset_index(drop=True, inplace=True )
#drop=True를 해야 기존 인덱스를 없애줌

In [11]:
X = df['title']+' '+df['summary']
Y= df['category']

In [12]:
len(X)
print(X[1250:1350])


1250    나쁜 와이프가 행복하다 결혼과 이혼의 허와 실뿐만 아니라, 도장 찍기 일보 직전의 ...
1251    카페에서 배우는 가정집 인테리어 조명부터 가구, 벽면, 공간 분할, 소품 활용까지 ...
1252    그렇게밖에 말 못해? 미국의 저명한 심리학 박사이자 부부치료 전문가인 낸시 드레퓌스...
1253    이상봉의 패션 이즈 패션 1980년 패션 디자이너로 데뷔해 현재까지 30년 넘게 한...
1254    구르메 빠라디 Gourmet Paradis 우리나라에도 미식문화가 자리를 잡아감에 ...
                                          ...                                    
1345    뷰티 스타일리스트 네일아트 북 이 책은 우리나라 네일산업의 급속한 발전으로 인해 네...
1346    실내.외 정원 디자인 주거공간 내의 베란다 정원부터 주택 정원, 업무공간과 상업공간...
1347    와인 가이드 & 홈메이드 와인 무작정 따라하기 와인의 역사와 문화, 와인을 즐기는 ...
1348    후루룩 한 컵 후다닥 한 그릇 신혼주부나 독신자들을 위한 1인 또는 2인 기준의 기...
1349    내 몸의 休 허브티 오렌지 필, 캐모마일, 레몬 밤, 로즈마리 등의 허브를 올컬러 ...
Length: 100, dtype: object


In [13]:
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)

['Art_Culture' 'Economic_Management' 'Health_Hobby_Leisure'
 'Home_Cook_Beauty' 'Novel_Poem' 'Religion_Mysticism' 'Science'
 'reference_book']


In [14]:
with open('./data/book_category_encoder.pickle', 'wb') as f:
  pickle.dump(encoder, f)

print(labeled_Y)

[3 3 3 ... 6 6 6]


In [15]:
onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [16]:
stopwords = pd.read_csv('./stopwords.csv')

In [18]:
okt = Okt()
print(type(X))

<class 'pandas.core.series.Series'>


In [19]:
okt_X = okt.morphs(X[0]) # 첫번째 title만 형태소 분리를 해서 okt_X에 넣어줌
print(X[0])
print(okt_X)

황성주 박사의 생식과 건강 생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레스테롤, 설탕을 많이함유하고 있는 요즘 음식은 건강을 해친다. 일반식에 비해 에너지 효율이 5,6배 높은 생식를 먹고 병이 나 은 임상결과를 다양한 사례와 함께 실었다.
['황', '성주', '박사', '의', '생식', '과', '건강', '생식', '을', '통한', '식생활', '개선', '으로', '성인병', '을', '치료', '하는', '방법', '서', '.', '육류', '등', '의', '포화지방', '이나', '콜레스테롤', ',', '설탕', '을', '많이', '함유', '하고', '있는', '요즘', '음식', '은', '건강', '을', '해친다', '.', '일반', '식', '에', '비해', '에너지', '효율', '이', '5', ',', '6', '배', '높은', '생', '식', '를', '먹고', '병', '이', '나', '은', '임', '상', '결과', '를', '다양한', '사례', '와', '함께', '실었다', '.']


In [20]:
for i in range(len(X)):
  X[i] = okt.morphs(X[i])
print(X)

0        [황, 성주, 박사, 의, 생식, 과, 건강, 생식, 을, 통한, 식생활, 개선, ...   
1        [아름다운, 청년, ,, 대니, 서의, 집, 환경, 운동, 가, 대니, 서가, 공개...  
2        [여왕, 패션, 종이, 인형, 다양한, 업적, 을, 가진, 세계, 의, 여왕, 패션... 
3        [더, 포스터, 북, by, 이미소, 직관, 적, 인, 제목, 처럼, 권, 마다, ...     
4        [가죽, 소품, 가죽공예, 의, 기본, 테크닉, 에서부터, 명함, 지갑, ,, 동전...
                                           ...                                    
61139    [파브르, 곤충기, 1, 1907년, 에, 완결, 된, <, 파브르, 곤충기, >,...       
61140    [처음, 만나는, 자동, 제어공학, IT, Cookbook, 한빛, 교재, 시리즈,...      
61141    [현산어보, 를, 찾아서, 1, --, 한국, 과학, 문화재단, ', 우수, 과학,...    
61142    [전염병, 의, 문, 화사, 인간, 을, 비롯, 한, 모든, 생명체, 는, 미생물,...  
61143    [세상, 을, 이루는, 모든, 원소, 118, 모든, 물질, 의, 기본, 인, 원소...    
Length: 61144, dtype: object


In [21]:
for i in range(len(X)):
  result = []
  for j in range(len(X[i])):
    if len(X[i][j]) > 1:
      if X[i][j] not in list(stopwords['stopword']):
        result.append(X[i][j])
  X[i] = ' '.join(result)
  
print(X)

0        성주 박사 생식 건강 생식 통한 식생활 개선 성인병 치료 방법 육류 포화지방 이나 ... 
1        아름다운 청년 대니 서의 환경 운동 대니 서가 공개 환경 꾸미기 교외 위치 부모님 ... 
2        여왕 패션 종이 인형 다양한 업적 가진 세계 여왕 패션 직접 입혀 아름답고 다양한 ... 
3        포스터 by 이미소 직관 제목 마다 포스터 작품 10 가득 채워진 도서 일반 종이 ...     
4        가죽 소품 가죽공예 기본 테크닉 에서부터 명함 지갑 동전 지갑 물림 지갑 파우치 안...
                                                ...                                        
61139    파브르 곤충기 1907년 완결 파브르 곤충기 10 방대함을 자랑 곤충 연구 문학자 ...     
61140    처음 만나는 자동 제어공학 IT Cookbook 한빛 교재 시리즈 362 자동 제어...           
61141    현산어보 찾아서 -- 한국 과학 문화재단 우수 과학 인증 도서 ', 시사저널 올해 ...    
61142    전염병 화사 인간 비롯 모든 생명체 미생물 비롯 되었다 인간 병원 미생물 역사 관계...
61143    세상 이루는 모든 원소 118 모든 물질 기본 원소 깔끔하게 직관 이해 있도록 비주...   
Length: 61144, dtype: object


In [22]:
token = Tokenizer()
token.fit_on_texts(X) # 어떤형태소를 무슨 숫자로 만들지 지정
tokened_X = token.texts_to_sequences(X) # 해당 형태소를 라벨링 된 숫자로 변경
print(tokened_X[0])

[17354, 412, 5350, 87, 5350, 866, 2262, 1098, 7019, 196, 8, 4761, 27403, 50, 6620, 3100, 308, 7201, 78, 87, 62270, 358, 1871, 583, 706, 299, 1114, 273, 13, 98, 571]


In [23]:
with open('./data/books_token.pickle', 'wb') as f:
  pickle.dump(token, f)

In [24]:
wordsize = len(token.word_index) +1 #나중에 0을 포함하려고
print(wordsize)

133252


In [25]:
max = 0
for i in range(len(tokened_X)):
  if max < len(tokened_X[i]):
    max = len(tokened_X[i])

print(max)

972


In [26]:
X_pad = pad_sequences(tokened_X, max) #토큰의 개수가 16개가 되게끔 알아서 앞쪽에 0을 붙여준다
print(X_pad[:10])

[[   0    0    0 ...   13   98  571]
 [   0    0    0 ...    2  411  707]
 [   0    0    0 ...  268 6245  539]
 ...
 [   0    0    0 ... 2834 1811 1556]
 [   0    0    0 ...  145  321 1423]
 [   0    0    0 ... 9737  672 1988]]


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size=0.15
)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(51972, 972)
(9172, 972)
(51972, 8)
(9172, 8)


In [28]:
xy = X_train, X_test, Y_train, Y_test
np.save('./data/books_data_max_{}_size_{}'.format(max,wordsize), xy)


C:\Users\tyler\anaconda3\envs\AI_exam\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
